In [1]:
import tensorflow as tf
import numpy as np
import os
import random

import matplotlib.pyplot as plt

from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg19 import preprocess_input

tfk = tf.keras
tfkl = tfk.layers

In [2]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [3]:
dataset_dir = '../input/ann-challenge-1-full/FullDataset'
labels = ['Apple','Blueberry','Cherry','Corn','Grape','Orange','Peach','Pepper','Potato','Raspberry','Soybean','Squash','Strawberry','Tomato']

In [4]:
batch_size = 256
img_height = 256
img_width = 256
channels = 3
input_shape = (img_width,img_height,channels)
epochs = 60
layers_frozen = 14

In [7]:
idg = ImageDataGenerator(
    zoom_range=0.33,
    horizontal_flip=True,
    vertical_flip=True,
    width_shift_range=55,
    height_shift_range=55,
    rotation_range=35,
    fill_mode='constant',
    cval=0,
    preprocessing_function=preprocess_input, 
    validation_split=0.2) 

In [8]:
train_data_generator = idg.flow_from_directory(
                  directory=dataset_dir, target_size=(img_width, img_height),
                  batch_size=batch_size, seed=seed, subset="training")
  
valid_data_generator = idg.flow_from_directory(
                  directory=dataset_dir, target_size=(img_width, img_height), 
                  batch_size=batch_size, seed=seed, subset="validation")

In [9]:
# Download and plot the VGG16 model
supernet = tfk.applications.VGG19(
    include_top=False,
    weights="imagenet",
    input_shape=(img_width,img_height,channels)
)
supernet.summary()

In [10]:
# Define the activation function
lrelu = lambda x: tf.keras.activations.relu(x, alpha=0.01)

# Use the supernet as feature extractor
supernet.trainable = False

inputs = tfk.Input(shape=(img_height,img_width,channels))
x = supernet(inputs)
x = tfkl.Flatten(name='Flattening')(x)
x = tfkl.Dropout(0.25, seed=seed)(x)
x = tfkl.Dense(
    256, 
    activation=lrelu,
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)
x = tfkl.Dropout(0.25, seed=seed)(x)
x = tfkl.Dense(
    512, 
    activation=lrelu,
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)
x = tfkl.Dropout(0.25, seed=seed)(x)
outputs = tfkl.Dense(
    14, 
    activation='softmax',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)


# Connect input and output through the Model class
fine_tuning_v2_model = tfk.Model(inputs=inputs, outputs=outputs, name='fine_tuning_v2_model')

# Compile the model
fine_tuning_v2_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')
fine_tuning_v2_model.summary()

In [11]:
# Set all VGG layers to True
fine_tuning_v2_model.get_layer('vgg19').trainable = True
for i, layer in enumerate(fine_tuning_v2_model.get_layer('vgg19').layers):
   print(i, layer.name, layer.trainable)

In [12]:
# Freeze first N layers, e.g., until 14th
for i, layer in enumerate(fine_tuning_v2_model.get_layer('vgg19').layers[:layers_frozen]):
  layer.trainable=False
for i, layer in enumerate(fine_tuning_v2_model.get_layer('vgg19').layers):
   print(i, layer.name, layer.trainable)
fine_tuning_v2_model.summary()

In [13]:
# Compile the model
fine_tuning_v2_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-4), metrics='accuracy')

In [14]:
# Train the model
history = fine_tuning_v2_model.fit(
    train_data_generator,
    batch_size = batch_size,
    epochs = epochs,
    validation_data = valid_data_generator,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=5, restore_best_weights=True)]
).history

In [15]:
fine_tuning_v2_model.save('fine_tuning_v2_model')

In [ ]:
# Plot the training
plt.figure(figsize=(15,5))
plt.plot(history['loss'], label='Training', alpha=.8, color='#ff7f0e')
plt.plot(history['val_loss'], label='Validation', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
plt.plot(history['accuracy'], label='Training', alpha=.8, color='#ff7f0e')
plt.plot(history['val_accuracy'], label='Validation', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

<a href="./fine_tuning_v2_model/variables/variables.data-00000-of-00001"> Download File </a>

In [ ]:
max(history['val_accuracy'])